In [2]:
import os

from scipy import ndimage
import math
import json
import torch
import numpy as np
import pandas as pd

import cv2
import matplotlib

from matplotlib import pyplot as plt

from tqdm import tqdm
from pathlib import Path
from dotenv import load_dotenv

from utils.yolo import get_teeth_ROI
from utils.edge import tooth_isolation
from utils.preprocess import get_labels_by_image, get_image_by_labels
from utils.preprocess import xyxy_reformat, rotate_bounding_boxes, rect_include_another

matplotlib.use('module://matplotlib_inline.backend_inline')
load_dotenv()


True

In [3]:
model = torch.hub.load(r'..\YOLO', 'custom', path=r'..\YOLO\weights\8-bound.pt', source='local')
data_dir = '..' / Path(os.getenv('DATASET_DIR'))


In [7]:
target_labels = ['caries', 'filling', 'endo', 'post', 'crown', 'R.R']
# df = pd.read_csv('label_TF.csv', index_col='filename')
image_labels_df = get_image_by_labels()
# image_labels_df = df[target_labels]
# image_labels_df = get_image_by_labels(target_labels)[target_labels]

image_labels_df['crown'].sum()


635

In [4]:
filename = image_labels_df.index[1]
filepath_image = data_dir / f'{filename}.jpg'
filepath_json = data_dir / f'{filename}.json'


In [5]:
# Image
# Inference
results = model(filepath_image)

teeth_roi = get_teeth_ROI(results)
teeth_roi_images = teeth_roi['images'][filename]
teeth_roi_split_teeth = teeth_roi['split_teeth']
teeth_roi_images


D:\Users\douli\anaconda3\envs\dentist-CV-main\lib\site-packages\yolov5\utils\plots.py:475: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xyxy = torch.tensor(xyxy).view(-1, 4)


[{'flag': 'upper',
  'number': 0,
  'tooth_position': 'left',
  'org_file_name': '00008026',
  'offset': array([698, 281]),
  'image': array([[[129, 129, 129],
          [130, 130, 130],
          [133, 133, 133],
          ...,
          [135, 135, 135],
          [133, 133, 133],
          [131, 131, 131]],
  
         [[127, 127, 127],
          [129, 129, 129],
          [133, 133, 133],
          ...,
          [138, 138, 138],
          [139, 139, 139],
          [139, 139, 139]],
  
         [[127, 127, 127],
          [132, 132, 132],
          [136, 136, 136],
          ...,
          [145, 145, 145],
          [145, 145, 145],
          [146, 146, 146]],
  
         ...,
  
         [[142, 142, 142],
          [143, 143, 143],
          [137, 137, 137],
          ...,
          [117, 117, 117],
          [120, 120, 120],
          [122, 122, 122]],
  
         [[142, 142, 142],
          [139, 139, 139],
          [130, 130, 130],
          ...,
          [115, 115, 115],
   

In [6]:
teeth_roi = get_teeth_ROI(results)
teeth_roi_images = teeth_roi['images'][filename]
teeth_roi_split_teeth = teeth_roi['split_teeth']
teeth_roi_images


[{'flag': 'upper',
  'number': 0,
  'tooth_position': 'left',
  'org_file_name': '00008026',
  'offset': array([698, 281]),
  'image': array([[[129, 129, 129],
          [130, 130, 130],
          [133, 133, 133],
          ...,
          [135, 135, 135],
          [133, 133, 133],
          [131, 131, 131]],
  
         [[127, 127, 127],
          [129, 129, 129],
          [133, 133, 133],
          ...,
          [138, 138, 138],
          [139, 139, 139],
          [139, 139, 139]],
  
         [[127, 127, 127],
          [132, 132, 132],
          [136, 136, 136],
          ...,
          [145, 145, 145],
          [145, 145, 145],
          [146, 146, 146]],
  
         ...,
  
         [[142, 142, 142],
          [143, 143, 143],
          [137, 137, 137],
          ...,
          [117, 117, 117],
          [120, 120, 120],
          [122, 122, 122]],
  
         [[142, 142, 142],
          [139, 139, 139],
          [130, 130, 130],
          ...,
          [115, 115, 115],
   

# One file check

In [7]:
image_labels = {}
labels = get_labels_by_image(filepath_json, target_labels)
for target_roi in teeth_roi_images:
    flag = target_roi['flag']
    target_roi_image = target_roi['image']
    tooth_position = target_roi['tooth_position']
    im_g = cv2.cvtColor(target_roi_image, cv2.COLOR_RGBA2GRAY)
    im_g_shape = np.array(np.array(im_g.shape)[[1, 0]])

    isolation_data = tooth_isolation(im_g, flag=flag, tooth_position=tooth_position, rotation_fix=False)
    if not isolation_data:
        continue
    regions = isolation_data['crop_regions']
    theta = isolation_data['angle']
    offset = target_roi['offset']

    phi = math.radians(theta)
    for label in labels:
        xyxy = np.hstack(label['points'])  # [x, y, x, y]
        xyxy = xyxy_reformat(np.array([xyxy]))

        xyxy = xyxy - np.tile(offset, 2)
        if xyxy.min() < 0:
            continue

        xyxy = rotate_bounding_boxes(phi, im_g_shape, xyxy)
        xyxy = xyxy[0].astype(int)

        for tooth_number, region in regions.items():
            tooth_xyxy = region['xyxy']
            key = f'{filename} {tooth_number}'
            if not key in image_labels.keys():
                image_labels[key] = []
            if rect_include_another(tooth_xyxy, xyxy) > 0.5:
                image_labels[key].append(label['label'])

image_labels


{'00008026 16': [],
 '00008026 15': [],
 '00008026 14': [],
 '00008026 12': [],
 '00008026 11': ['crown'],
 '00008026 21': ['endo'],
 '00008026 22': ['post', 'crown'],
 '00008026 50': ['post', 'endo', 'crown'],
 '00008026 51': ['crown'],
 '00008026 24': [],
 '00008026 25': [],
 '00008026 26': ['crown'],
 '00008026 46': [],
 '00008026 45': ['caries'],
 '00008026 44': [],
 '00008026 42': [],
 '00008026 41': [],
 '00008026 31': [],
 '00008026 32': [],
 '00008026 34': [],
 '00008026 35': [],
 '00008026 36': []}

In [8]:
import warnings

warnings.filterwarnings("ignore")

image_labels = {}
for filename in tqdm(image_labels_df.index):
    filepath_image = data_dir / f'{filename}.jpg'
    filepath_json = data_dir / f'{filename}.json'

    results = model(filepath_image)

    teeth_roi = get_teeth_ROI(results)
    teeth_roi_images = teeth_roi['images'][filename]
    teeth_roi_split_teeth = teeth_roi['split_teeth']

    labels = get_labels_by_image(filepath_json, target_labels)
    for target_roi in teeth_roi_images:
        target_roi_image = target_roi['image']
        flag = target_roi['flag']
        tooth_position = target_roi['tooth_position']

        im_g = cv2.cvtColor(target_roi_image, cv2.COLOR_RGBA2GRAY)
        im_g_shape = np.array(np.array(im_g.shape)[[1, 0]])

        isolation_data = tooth_isolation(im_g, flag=flag, tooth_position=tooth_position, rotation_fix=False, save=True,
                                         filename=filename)
        if not isolation_data:
            continue
        regions = isolation_data['crop_regions']
        theta = isolation_data['angle']
        offset = target_roi['offset']

        phi = math.radians(theta)
        for label in labels:
            xyxy = np.hstack(label['points'])  # [x, y, x, y]
            xyxy = xyxy_reformat(np.array([xyxy]))

            xyxy = xyxy - np.tile(offset, 2)
            if xyxy.min() < 0:
                continue

            xyxy = rotate_bounding_boxes(phi, im_g_shape, xyxy)
            xyxy = xyxy[0].astype(int)

            for tooth_number, region in regions.items():
                tooth_xyxy = region['xyxy']
                key = f'{filename} {tooth_number}'
                if not key in image_labels.keys():
                    image_labels[key] = []
                if rect_include_another(tooth_xyxy, xyxy) > 0.5:
                    image_labels[key].append(label['label'])


100%|██████████| 600/600 [16:05<00:00,  1.61s/it]


In [9]:
target_roi


{'flag': 'lower',
 'number': 2,
 'tooth_position': 'right',
 'org_file_name': '202012051012169372006_0690223A',
 'offset': array([1429,  622]),
 'image': array([[[ 51,  51,  51],
         [ 51,  51,  51],
         [ 50,  50,  50],
         ...,
         [ 44,  44,  44],
         [ 45,  45,  45],
         [ 46,  46,  46]],
 
        [[ 50,  50,  50],
         [ 50,  50,  50],
         [ 49,  49,  49],
         ...,
         [ 43,  43,  43],
         [ 44,  44,  44],
         [ 45,  45,  45]],
 
        [[ 49,  49,  49],
         [ 49,  49,  49],
         [ 49,  49,  49],
         ...,
         [ 42,  42,  42],
         [ 44,  44,  44],
         [ 44,  44,  44]],
 
        ...,
 
        [[124, 124, 124],
         [125, 125, 125],
         [127, 127, 127],
         ...,
         [ 83,  83,  83],
         [ 83,  83,  83],
         [ 83,  83,  83]],
 
        [[121, 121, 121],
         [123, 123, 123],
         [124, 124, 124],
         ...,
         [ 85,  85,  85],
         [ 86,  86,  8

In [10]:
# for tooth_number, r in region.items():
#     print(r)
# target_roi
# label
# !jupyter nbconvert --to script prepare_classify_format.ipynb

j = json.dumps(image_labels)

with open('image_labels_for_classify.json', 'w') as f:
    f.write(j)


## Check crop and origin image annotation count.

In [22]:
crop_annotation_number = {}
for k, v in image_labels.items():
    filename, tooth_number = k.split(' ')

    if filename not in crop_annotation_number.keys():
        crop_annotation_number[filename] = {i: 0 for i in target_labels}

    for i in v:
        crop_annotation_number[filename][i] += 1

counter_diff = {}
for filename, counter in crop_annotation_number.items():
    counter_diff[filename] = {}

    org_number_row = image_labels_df.loc[filename]
    for k in target_labels:
        counter_diff[filename][k] = org_number_row.loc[k] - counter[k]

counter_diff_df = pd.DataFrame.from_dict(counter_diff, orient='index')
counter_diff_df.loc[(counter_diff_df > 2).any(axis=1)]



,caries,filling,endo,post,crown,R.R
00008113,2,7,2,1,1,0
00008140,3,2,1,0,0,0
00008152,0,0,3,4,1,0
00008154,0,2,8,4,7,1
00008157,0,1,0,0,3,0
...,...,...,...,...,...,...
202012031112019181962_0270118A,0,3,5,1,1,1
202012040312239241975_0820101A,0,0,4,1,0,0
202012050312249422012_0660508A,0,1,6,1,1,0
202012050312469402009_0681119A,0,3,1,0,1,0
